In [32]:
import concurrent.futures
from itertools import product

import numpy as np
import pandas as pd
from tqdm import tqdm

from noisyreach.deviation import AVAIL_SYSTEMS, deviation, get_max_diam

In [ ]:
accuracy = [0.9, 0.9, 1, 1, 1]
np.max(deviation(0.3, accuracy, system="CAR", plotting=True))

In [ ]:
systems = AVAIL_SYSTEMS
latency = np.linspace(0.001, 0.020, 20)
accuracy = np.linspace(0.5, 1.0, 20)

rows = []

# Example usage of maximum diameter calculation
# diameter = get_max_diam(0.02, (1 - 0.8), "CAR")

with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures_to_args = {}
    for argset in product(systems, latency, accuracy):
        s, L, a = argset
        futures_to_args[executor.submit(get_max_diam, L, 1 - a, s)] = argset

    print(argset)
    with tqdm(total=len(futures_to_args)) as pbar:
        for future in concurrent.futures.as_completed(futures_to_args):
            try:
                diameter = future.result()
                rows.append([*futures_to_args[future], diameter])
                pbar.update(1)
            except AttributeError as e:
                print(e)
                print(futures_to_args[future])

df = pd.DataFrame(rows, columns=["system", "latency", "accuracy", "diameter"])
df